Un DataFrame PySpark  équivaut à une table relationnelle dans Spark SQL et peut être créé à l'aide de diverses fonctions dans SparkSession :

## Why PySpark Dataframe

**RDD**-   Lorsque vous souhaitez une transformation et des actions de bas niveau, nous utilisons des RDD. De plus, lorsque nous avons besoin d'abstractions de haut niveau, nous utilisons des RDD.
Lors de l'exécution d'opérations simples de regroupement et d'agrégation, l'API RDD est plus lente. 
Depuis les API RDD, un utilisateur doit définir le schéma manuellement.

**DataFrame** -   Lorsque nous avons besoin d'un haut niveau d'abstraction et pour les données non structurées, telles que les flux multimédias ou les flux de texte.En effectuant une analyse exploratoire, en créant des statistiques agrégées sur les données, les dataframes sont plus rapides.Dans dataframe, il n'est pas nécessaire de spécifier un schéma. Généralement, il découvre le schéma automatiquement.

En réalité, grossièrement les RDD sont devenus obsolètes. Et la raison derrière cela est qu'au fur et à mesure que Spark est devenu mature, il a commencé à ajouter des fonctionnalités pour le  data warehousing, le big data analytics, et la data science.

Pour ce faire, il a introduit les DataFrames PySpark

In [4]:
from pyspark import SparkContext, HiveContext, SparkConf 

In [5]:
conf = SparkConf().setMaster('local[*]')                # setMaster('yarn-client')
conf = conf.setAppName('my_second_app')
conf =  conf.set('spark.ui.port', '5050')               # definition du uri spark
conf= conf.set('spark.sql.shuffle.partitions','4')
sc = SparkContext(conf=conf)                            # Instanciation du context spark
hctx = HiveContext(sc)                                  # Instanciation du context hive

In [6]:
df = hctx.sql("USE Upec_2022")

In [7]:
#Recuperation de données
query = "SELECT * FROM iad_stb_model"

#Execution de la requête
dtf = hctx.sql(query)

In [8]:
# dtf est un dataframe
print(dtf.printSchema())

root
 |-- dat_phto: string (nullable = true)
 |-- idnt_comp_serv: long (nullable = true)
 |-- rdn_samp: integer (nullable = true)
 |-- anc_actv: integer (nullable = true)
 |-- ref_date: string (nullable = true)
 |-- dat_rsil_cds: string (nullable = true)
 |-- flg_rsil: integer (nullable = true)
 |-- val_debit_down: float (nullable = true)
 |-- libl_res: string (nullable = true)
 |-- libl_tech: string (nullable = true)
 |-- nb_iad: integer (nullable = true)
 |-- iad_anc_info: integer (nullable = true)
 |-- iad_userkey2: string (nullable = true)
 |-- fdate_iad_anc: integer (nullable = true)
 |-- iad_model: string (nullable = true)
 |-- iad_vendor: string (nullable = true)
 |-- iad_softwarever: string (nullable = true)
 |-- rk_asc_iad_conf: integer (nullable = true)
 |-- nb_stv: integer (nullable = true)
 |-- stb_anc_info: integer (nullable = true)
 |-- fdate_stb_anc: integer (nullable = true)
 |-- stb_model: string (nullable = true)
 |-- stb_vendor: string (nullable = true)
 |-- stb_soft

In [15]:
dtf.show(5)

+--------------+--------+--------+------------+--------+--------------+--------+---------+------+------------+------------+-------------+---------+----------+--------------------+---------------+------+------------+-------------+--------------+----------+---------------+---------------+----------+---------+
|idnt_comp_serv|rdn_samp|anc_actv|dat_rsil_cds|flg_rsil|val_debit_down|libl_res|libl_tech|nb_iad|iad_anc_info|iad_userkey2|fdate_iad_anc|iad_model|iad_vendor|     iad_softwarever|rk_asc_iad_conf|nb_stv|stb_anc_info|fdate_stb_anc|     stb_model|stb_vendor|stb_softwarever|rk_asc_stb_conf|noinadinfo|nostbinfo|
+--------------+--------+--------+------------+--------+--------------+--------+---------+------+------------+------------+-------------+---------+----------+--------------------+---------------+------+------------+-------------+--------------+----------+---------------+---------------+----------+---------+
|  610002561962|       0|      26|           ?|       0|       20000.0|  

## Traitement des données

Classes **pyspark.sql.function**  permettent de faire des réquètes Spark SQL et DataFrames

In [9]:
from pyspark.sql import functions as F

Supprimer des colonnes : **drop**

In [12]:
dtf = dtf.drop(dtf.dat_phto)
dtf = dtf.drop(dtf.ref_date)

Modifier des colonnes  : **withColumn** , **when**

In [13]:
dtf = dtf.withColumn("flg_rsil", F.when(dtf.dat_rsil_cds=='?',0).otherwise(1))

Filtrer des colonnes / Sélectionner des colonnes : **select**, **SelectExpr**, **where**


In [15]:
#dtf2 = dtf.select(dtf.idnt_comp_serv, dtf.flg_rsil, dtf.rdn_samp).where("rdn_samp=1")
dtf2 = dtf.select('idnt_comp_serv', 'flg_rsil', 'rdn_samp').where("rdn_samp=1")
dtf2.show(5)

+--------------+--------+--------+
|idnt_comp_serv|flg_rsil|rdn_samp|
+--------------+--------+--------+
|  610004644264|       0|       1|
|  610001492507|       0|       1|
|  610004556803|       0|       1|
|  610002465916|       0|       1|
|  610001621486|       0|       1|
+--------------+--------+--------+
only showing top 5 rows



In [16]:
# Ajouter un label au nom de la colonne et l'afficher
dtf.selectExpr('idnt_comp_serv AS chpA', 'flg_rsil').where("rdn_samp=1 AND flg_rsil=0").show(5)

+------------+--------+
|        chpA|flg_rsil|
+------------+--------+
|610004644264|       0|
|610001492507|       0|
|610004556803|       0|
|610002465916|       0|
|610001621486|       0|
+------------+--------+
only showing top 5 rows



In [27]:
dtf2.registerTempTable('tmp_table')

In [29]:
hctx.sql("SHOW tables").show()

+-------------+-----------+
|    tableName|isTemporary|
+-------------+-----------+
|iad_stb_model|      false|
|     svi_data|      false|
| svi_data_orc|      false|
| svi_data_pqt|      false|
|    tmp_table|       true|
+-------------+-----------+



Faire des jointures **JOIN**

In [19]:
dtf.join(dtf2, ((dtf.idnt_comp_serv == dtf2.idnt_comp_serv) & (dtf.rdn_samp == dtf2.rdn_samp)), "inner").show(3)

+--------------+--------+--------+------------+--------+--------------+--------+---------+------+------------+------------+-------------+---------+----------+--------------------+---------------+------+------------+-------------+--------------+----------+---------------+---------------+----------+---------+--------------+--------+--------+
|idnt_comp_serv|rdn_samp|anc_actv|dat_rsil_cds|flg_rsil|val_debit_down|libl_res|libl_tech|nb_iad|iad_anc_info|iad_userkey2|fdate_iad_anc|iad_model|iad_vendor|     iad_softwarever|rk_asc_iad_conf|nb_stv|stb_anc_info|fdate_stb_anc|     stb_model|stb_vendor|stb_softwarever|rk_asc_stb_conf|noinadinfo|nostbinfo|idnt_comp_serv|flg_rsil|rdn_samp|
+--------------+--------+--------+------------+--------+--------------+--------+---------+------+------------+------------+-------------+---------+----------+--------------------+---------------+------+------------+-------------+--------------+----------+---------------+---------------+----------+---------+--------

Calculer des aggrégats : **Groupby**, **count**

In [33]:
# Calculer des aggrégats
dtf.groupBy('flg_rsil').count().show() 

+--------+------+
|flg_rsil| count|
+--------+------+
|       0|601449|
|       1|  8495|
+--------+------+



In [34]:
dtf.groupBy('flg_rsil','libl_tech').count().show() 

+--------+---------+------+
|flg_rsil|libl_tech| count|
+--------+---------+------+
|       0|        ?|  4698|
|       0|     FTTH|  3375|
|       1|        ?|   373|
|       0|     xDSL|509593|
|       0|      THD| 83783|
|       1|      THD|  1170|
|       1|     xDSL|  6910|
|       1|     FTTH|    42|
+--------+---------+------+



In [35]:
dtf.groupBy(dtf.flg_rsil, dtf.libl_tech).count().show() 

+--------+---------+------+
|flg_rsil|libl_tech| count|
+--------+---------+------+
|       0|        ?|  4698|
|       0|     FTTH|  3375|
|       1|        ?|   373|
|       0|     xDSL|509593|
|       0|      THD| 83783|
|       1|      THD|  1170|
|       1|     xDSL|  6910|
|       1|     FTTH|    42|
+--------+---------+------+



Calcule de covariance, corrélation entre 2 colonnes **cov, corr**

In [20]:
# Le format de val_debit_down n'est pas entier
dtf.selectExpr('flg_rsil', 'CAST(val_debit_down AS INT) as val_debit_down').\
where(dtf.rdn_samp == 1).cov("flg_rsil", "val_debit_down")

-279.37907793573163

In [21]:
dtf.selectExpr('flg_rsil', 'CAST(val_debit_down AS INT) as val_debit_down').\
where(dtf.rdn_samp == 1).corr("flg_rsil", "val_debit_down")

-0.014757322753845652

 **FreqItem** : éléments qui apparaissent plus de X % de la colonne

In [22]:
# selection des modalités ayant plus de 50%
dtf.stat.freqItems(["rdn_samp", "anc_actv", "dat_rsil_cds"], 0.50).show(truncate=False)

+------------------+------------------+----------------------+
|rdn_samp_freqItems|anc_actv_freqItems|dat_rsil_cds_freqItems|
+------------------+------------------+----------------------+
|[1, 0]            |[4, 39]           |[26/02/2015, ?]       |
+------------------+------------------+----------------------+



Tableau croisé à deux entrée **crosstab**

In [41]:
dtf.crosstab("flg_rsil", "libl_res").show()

+-----------------+----+------+------+-----+-----+------+-----+
|flg_rsil_libl_res|   ?|Axione|    BT|BT_HZ|   FT|    NC|  NCC|
+-----------------+----+------+------+-----+-----+------+-----+
|                1| 373|   219|  4025|   42|  690|  1976| 1170|
|                0|4698| 16527|310830| 3375|38488|143748|83783|
+-----------------+----+------+------+-----+-----+------+-----+



Eliminations des doublons  avec **dropDuplicates**

In [23]:
# doublons parfaits
dtf.dropDuplicates().count()

609935

In [24]:
# doublons sur des colonnes définies
dtf.dropDuplicates(['flg_rsil', 'rdn_samp'])

DataFrame[idnt_comp_serv: bigint, rdn_samp: int, anc_actv: int, dat_rsil_cds: string, flg_rsil: int, val_debit_down: float, libl_res: string, libl_tech: string, nb_iad: int, iad_anc_info: int, iad_userkey2: string, fdate_iad_anc: int, iad_model: string, iad_vendor: string, iad_softwarever: string, rk_asc_iad_conf: int, nb_stv: int, stb_anc_info: int, fdate_stb_anc: int, stb_model: string, stb_vendor: string, stb_softwarever: string, rk_asc_stb_conf: int, noinadinfo: int, nostbinfo: int]

Valeurs manquantes : **na**

In [46]:
dtf.dropna().count()

381590

## Python UDF avec Pyspark DataFrame

In [ ]:
UDF, veut dire Fonctions définies par l’utilisateur. 
L’UDF nous permettra d’appliquer les fonctions directement dans les dataframes et les bases de données SQL en python. 
Autrement une fonction python n'est pas directement utilisable sur PySpark avec un simple Apply. 
En pyspark , UDF est la solution

In [28]:
import re

def removeCar(str):
    return re.sub("[a-zA-Z]", "", str, 0)

def removeDig(str):
    return re.sub("\d", "", str, 0)

def removeSpe(str):
    return re.sub("\W", "", str, 0)

Definition de  UDF comme data type

In [29]:
from pyspark.sql.types import *

schema = StringType()

from pyspark.sql.functions import udf

UDFremoveDig = udf(removeDig, schema)
UDFremoveCar = udf(removeCar, schema)
UDFremoveSpe = udf(removeSpe, schema)

Utilisation de L'UDF avec withColumn (PySpark DataFrame)

In [30]:
dtf.withColumn("noDigit", UDFremoveDig("iad_softwarever"))\
    .withColumn("noCar", UDFremoveCar("iad_softwarever"))\
    .withColumn("noSpe", UDFremoveSpe("iad_softwarever"))\
    .select("iad_softwarever", "noDigit", "noCar", "noSpe").show(truncate=False)

+-------------------------------------+---------------------+-------------------------+--------------------------------+
|iad_softwarever                      |noDigit              |noCar                    |noSpe                           |
+-------------------------------------+---------------------+-------------------------+--------------------------------+
|B400FC_RECO_7.6.22-B40004_MAIN_9.1.50|BFC_RECO_..-B_MAIN_..|400__7.6.22-40004__9.1.50|B400FC_RECO_7622B40004_MAIN_9150|
|5B10I4-5B00IU                        |BI-BIU               |5104-500                 |5B10I45B00IU                    |
|B11001_MAIN_8.6.62                   |B_MAIN_..            |11001__8.6.62            |B11001_MAIN_8662                |
|B11001_MAIN_8.6.64                   |B_MAIN_..            |11001__8.6.64            |B11001_MAIN_8664                |
|B400FC_RECO_7.6.22-B40004_MAIN_9.1.50|BFC_RECO_..-B_MAIN_..|400__7.6.22-40004__9.1.50|B400FC_RECO_7622B40004_MAIN_9150|
|B400FC_RECO_7.6.22-B40004_MAIN_

## Customization de functions python avec SPARK SQL

In [31]:
hctx.registerFunction("UDFremoveCar",lambda x : removeCar(x), StringType())

In [32]:
df = hctx.sql("SELECT iad_softwarever, UDFremoveCar(iad_softwarever) as noCar FROM iad_stb_model LIMIT 100").show(1)

+--------------------+--------------------+
|     iad_softwarever|               noCar|
+--------------------+--------------------+
|B400FC_RECO_7.6.2...|400__7.6.22-40004...|
+--------------------+--------------------+
only showing top 1 row

